In [25]:
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langserve import RemoteRunnable
from langchain_core.runnables import RunnableParallel
import bs4
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.documents import Document
from typing import List, Union
import re
from langchain_upstage import UpstageLayoutAnalysisLoader
import os
import copy

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

# 로드

In [24]:
from langchain_upstage import UpstageLayoutAnalysisLoader
import os

api_key = os.getenv('UPSTAGE_API_KEY')
file_path = "../assets/230517 (보도자료) AI 대응전략 세미나 개최.pdf"

# For image files, set use_ocr to True to perform OCR inference on the document before layout detection.
loader = UpstageLayoutAnalysisLoader(file_path
                                     , split="page"
                                     , api_key=api_key
                                     , use_ocr=True)

# For improved memory efficiency, consider using the lazy_load method to load documents page by page.
pages = loader.load()  # or loader.lazy_load()
 
for page in pages:
    print(page) # print the document content

page_content='<table id='0' style='font-size:14px'><tr><td>금융위원회</td><td>보도자료</td><td></td><td></td><td>다시 대한민국! 새로운 국민의 나라</td><td></td><td></td><td></td></tr><tr><td>보도시점</td><td></td><td>2023. 5. 17.(수)</td><td>14:00</td><td>배포</td><td>2023.</td><td>5. 17.(수)</td><td>11:00</td></tr></table> <br><h1 id='1' style='font-size:22px'>금융권, 금융 AI 경쟁력 강화 방안 모색</h1> <p id='2' data-category='paragraph' style='font-size:20px'>- 신정원, 「금융 산업의 인공지능 대응전략 세미나」 개최<br>- 금융위, ①양질의 빅데이터 확보, ②금융 데이터 관련 규제혁신,<br>③ 금융 AI 신뢰 제고 등 지원</p> <p id='3' data-category='paragraph' style='font-size:20px'>5월 17일, 한국신용정보원이 주최, 금융위원회가 후원하는 「금융 산업의<br>인공지능 대응전략 세미나」가 개최되었다. 이번 간담회에서 금융위원회,<br>금융회사, 관계 전문가 등은 금융분야의 AI 활용 현황을 공유하고, 금융<br>산업의 경쟁력을 향상시키기 위한 AI 정책 방향 등에 대해 논의하였다.</p> <p id='4' data-category='paragraph' style='font-size:16px'>< 「금융 산업의 인공지능 대응전략 세미나」 개요 ></p> <p id='5' data-category='list' style='font-size:14px'>■ 일시 / 장소 : '23.5.17일(수) 14:00~ 16:00 / 은행회관 뱅커스클럽<br>■ 세미나 내용</p> <br><h1 id='6' style='font-size:

In [27]:
def pretty_print(docs: List[Document]):
    # 결과 출력
    for doc in docs:
        print()
        print(f"Page {doc.metadata['page']} ===================================================")
        # print(f"Content: {doc.page_content[:200]}...")  # 일부만 출력
        print(f"Content: {doc.page_content}")  # 일부만 출력
        print(f"Metadata: {doc.metadata}")

pretty_print(pages)


Page 1 ===================================================
Content: <table id='0' style='font-size:14px'><tr><td>금융위원회</td><td>보도자료</td><td></td><td></td><td>다시 대한민국! 새로운 국민의 나라</td><td></td><td></td><td></td></tr><tr><td>보도시점</td><td></td><td>2023. 5. 17.(수)</td><td>14:00</td><td>배포</td><td>2023.</td><td>5. 17.(수)</td><td>11:00</td></tr></table> <br><h1 id='1' style='font-size:22px'>금융권, 금융 AI 경쟁력 강화 방안 모색</h1> <p id='2' data-category='paragraph' style='font-size:20px'>- 신정원, 「금융 산업의 인공지능 대응전략 세미나」 개최<br>- 금융위, ①양질의 빅데이터 확보, ②금융 데이터 관련 규제혁신,<br>③ 금융 AI 신뢰 제고 등 지원</p> <p id='3' data-category='paragraph' style='font-size:20px'>5월 17일, 한국신용정보원이 주최, 금융위원회가 후원하는 「금융 산업의<br>인공지능 대응전략 세미나」가 개최되었다. 이번 간담회에서 금융위원회,<br>금융회사, 관계 전문가 등은 금융분야의 AI 활용 현황을 공유하고, 금융<br>산업의 경쟁력을 향상시키기 위한 AI 정책 방향 등에 대해 논의하였다.</p> <p id='4' data-category='paragraph' style='font-size:16px'>< 「금융 산업의 인공지능 대응전략 세미나」 개요 ></p> <p id='5' data-category='list' style='font-size:14px'>■ 일시 / 장소 : '23.5.17일(수) 14:00~ 16:00 / 은행회관

# 전처리 (html to markdown)

In [28]:
from bs4 import BeautifulSoup
from markdownify import markdownify as md

def html_to_markdown(html_content):
    # BeautifulSoup을 사용하여 HTML 파싱
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # 불필요한 요소 제거 (예: 스크립트, 스타일)
    for script in soup(["script", "style"]):
        script.decompose()
    
    # 특정 요소 전처리 (필요한 경우)
    # 예: 특정 클래스를 가진 div를 제거하거나 수정
    # for div in soup.find_all("div", class_="unwanted-class"):
    #     div.decompose()
    
    # markdownify를 사용하여 HTML을 Markdown으로 변환
    markdown = md(str(soup), heading_style="ATX", strip=['a'])
    
    return markdown.strip()


# pages 리스트를 markdown_pages로 복사
markdown_pages = pages

# 각 페이지의 page_content를 마크다운으로 변환하여 markdown_pages에 업데이트
for i, page in enumerate(pages):
    markdown_page = html_to_markdown(page.page_content)
    markdown_pages[i].page_content = markdown_page  # 변환된 내용을 page_content에 저장

In [29]:
pretty_print(markdown_pages)


Page 1 ===================================================
Content: | 금융위원회 | 보도자료 |  |  | 다시 대한민국! 새로운 국민의 나라 |  |  |  |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 보도시점 |  | 2023\. 5\. 17\.(수) | 14:00 | 배포 | 2023\. | 5\. 17\.(수) | 11:00 |

   
# 금융권, 금융 AI 경쟁력 강화 방안 모색

 \- 신정원, 「금융 산업의 인공지능 대응전략 세미나」 개최  
\- 금융위, ①양질의 빅데이터 확보, ②금융 데이터 관련 규제혁신,  
③ 금융 AI 신뢰 제고 등 지원

 5월 17일, 한국신용정보원이 주최, 금융위원회가 후원하는 「금융 산업의  
인공지능 대응전략 세미나」가 개최되었다. 이번 간담회에서 금융위원회,  
금융회사, 관계 전문가 등은 금융분야의 AI 활용 현황을 공유하고, 금융  
산업의 경쟁력을 향상시키기 위한 AI 정책 방향 등에 대해 논의하였다.

 \< 「금융 산업의 인공지능 대응전략 세미나」 개요 \>

 ■ 일시 / 장소 : '23\.5\.17일(수) 14:00\~ 16:00 / 은행회관 뱅커스클럽  
■ 세미나 내용

   
# ① 발제

   
\- 금융산업의 인공지능 활용과 정책과제 (금융연구원 서정호 부원장)

   
# ② 주제발표

 \- 금융회사의 AI 대응현황 및 대응전략 : 차세대 AI 기술을 중심으로  
(우리은행 디지털전략그룹 옥일진 부행장)  
\- AI 및 로보어드바이저를 활용한 투자전략 (크래프트테크놀로지스 김형식 대표이사)  
\- 책임있는 AI를 구축하기 위한 마이크로소프트 AI 소개  
(한국마이크로소프트 김형석 DATA\&AI Lead)

   
# ③ 사례발표

 \- 신용정보를 활용한 AI 모형 기반 개인사업자 부도율 예측방법  
(신용정보원 홍동숙 IT보안팀장)  
\- 합성데이터(Synthetic

# 전처리 (기타 전처리)

In [31]:
preprocessed_docs = copy.deepcopy(pages)

for i, doc in enumerate(preprocessed_docs):
    # 과도한 줄바꿈 제거
    # 정규식을 사용하여 \n과 \n 사이에 공백이 있는 패턴을 \n\n으로 변경
    # \n\s*\n 패턴은 \n, 그 사이에 0개 이상의 공백(\s*)을 포함한 또 다른 \n을 찾음
    preprocessed_docs[i].page_content = re.sub(r'\n\s*\n', '\n\n', preprocessed_docs[i].page_content)
    preprocessed_docs[i].page_content = preprocessed_docs[i].page_content.replace("\n\n", "\n")
    preprocessed_docs[i].page_content = preprocessed_docs[i].page_content.replace("\n\n", "\n")
    preprocessed_docs[i].page_content = preprocessed_docs[i].page_content.replace("\n\n", "\n")
    preprocessed_docs[i].page_content = preprocessed_docs[i].page_content.replace("\n\n", "\n")
    preprocessed_docs[i].page_content = preprocessed_docs[i].page_content.replace("\n\n", "\n")
    preprocessed_docs[i].page_content = preprocessed_docs[i].page_content.replace("\n\n", "\n")
    
    # 기타 치환해야 할 것들
    preprocessed_docs[i].page_content = preprocessed_docs[i].page_content.replace("\-", "-")
    preprocessed_docs[i].page_content = preprocessed_docs[i].page_content.replace("\\", "")
    
pretty_print(preprocessed_docs)


Page 1 ===================================================
Content: | 금융위원회 | 보도자료 |  |  | 다시 대한민국! 새로운 국민의 나라 |  |  |  |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 보도시점 |  | 2023. 5. 17.(수) | 14:00 | 배포 | 2023. | 5. 17.(수) | 11:00 |
# 금융권, 금융 AI 경쟁력 강화 방안 모색
 - 신정원, 「금융 산업의 인공지능 대응전략 세미나」 개최  
- 금융위, ①양질의 빅데이터 확보, ②금융 데이터 관련 규제혁신,  
③ 금융 AI 신뢰 제고 등 지원
 5월 17일, 한국신용정보원이 주최, 금융위원회가 후원하는 「금융 산업의  
인공지능 대응전략 세미나」가 개최되었다. 이번 간담회에서 금융위원회,  
금융회사, 관계 전문가 등은 금융분야의 AI 활용 현황을 공유하고, 금융  
산업의 경쟁력을 향상시키기 위한 AI 정책 방향 등에 대해 논의하였다.
 < 「금융 산업의 인공지능 대응전략 세미나」 개요 >
 ■ 일시 / 장소 : '23.5.17일(수) 14:00~ 16:00 / 은행회관 뱅커스클럽  
■ 세미나 내용
# ① 발제
- 금융산업의 인공지능 활용과 정책과제 (금융연구원 서정호 부원장)
# ② 주제발표
 - 금융회사의 AI 대응현황 및 대응전략 : 차세대 AI 기술을 중심으로  
(우리은행 디지털전략그룹 옥일진 부행장)  
- AI 및 로보어드바이저를 활용한 투자전략 (크래프트테크놀로지스 김형식 대표이사)  
- 책임있는 AI를 구축하기 위한 마이크로소프트 AI 소개  
(한국마이크로소프트 김형석 DATA&AI Lead)
# ③ 사례발표
 - 신용정보를 활용한 AI 모형 기반 개인사업자 부도율 예측방법  
(신용정보원 홍동숙 IT보안팀장)  
- 합성데이터(Synthetic Data)를 통한 금융 AI 활성화 방안  
(신용정보원 허용준 선임조사역)
 # 4 패널